In [66]:
!find /content/test_data_v2 -maxdepth 3 -type f | head


/content/test_data_v2/test_data_v2/050bc88938bd4e739d075269fb2da9b4.jpg
/content/test_data_v2/test_data_v2/fea6699b3e5e487da4cb28b68fcd476a.jpg
/content/test_data_v2/test_data_v2/1bc52a6203c74518800823a4df70906e.jpg
/content/test_data_v2/test_data_v2/f980c9c9d1b6443e95cf43b6ef245dfd.jpg
/content/test_data_v2/test_data_v2/c1e29ca8e0b24dd8bdb1f09cd05eaf6e.jpg
/content/test_data_v2/test_data_v2/dc05fc8797e9408d86529a58f1a4b1c6.jpg
/content/test_data_v2/test_data_v2/c992a6424951495390fd5f90ac9e8672.jpg
/content/test_data_v2/test_data_v2/40e5cbd3c4c643c79c80c2fd7eaab876.jpg
/content/test_data_v2/test_data_v2/209a710959ab4a0bb4f760550b546ab3.jpg
/content/test_data_v2/test_data_v2/2c39356dd6534232b52f6b5a0a4254ac.jpg


In [67]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os


In [68]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/Stat_362_final_project/train.csv")
test  = pd.read_csv("/content/drive/MyDrive/Stat_362_final_project/test.csv")

print(train.columns)
print(test.columns)

print(train.head())
print(test.head())


Index(['Unnamed: 0', 'file_name', 'label'], dtype='object')
Index(['id'], dtype='object')
   Unnamed: 0                                        file_name  label
0           0  train_data/a6dcb93f596a43249135678dfcfc17ea.jpg      1
1           1  train_data/041be3153810433ab146bc97d5af505c.jpg      0
2           2  train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg      1
3           3  train_data/8542fe161d9147be8e835e50c0de39cd.jpg      0
4           4  train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg      1
                                                  id
0  test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg
1  test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg
2  test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg
3  test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg
4  test_data_v2/a16495c578b7494683805484ca27cf9f.jpg


In [69]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/Stat_362_final_project/train.csv")
test  = pd.read_csv("/content/drive/MyDrive/Stat_362_final_project/test.csv")

train['filepath'] = train['file_name'].apply(lambda x: "/content/train_data/" + x)
test['filepath'] = test['id'].apply(
    lambda x: "/content/test_data_v2/" + x
)




train.head(), test.head()


(   Unnamed: 0                                        file_name  label  \
 0           0  train_data/a6dcb93f596a43249135678dfcfc17ea.jpg      1   
 1           1  train_data/041be3153810433ab146bc97d5af505c.jpg      0   
 2           2  train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg      1   
 3           3  train_data/8542fe161d9147be8e835e50c0de39cd.jpg      0   
 4           4  train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg      1   
 
                                             filepath  
 0  /content/train_data/train_data/a6dcb93f596a432...  
 1  /content/train_data/train_data/041be3153810433...  
 2  /content/train_data/train_data/615df26ce9494e5...  
 3  /content/train_data/train_data/8542fe161d9147b...  
 4  /content/train_data/train_data/5d81fa12bc3b4ce...  ,
                                                   id  \
 0  test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg   
 1  test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg   
 2  test_data_v2/eb364dd2dfe34feda0e52466b7ce7

In [70]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    train,
    test_size=0.2,
    random_state=42,
    stratify=train['label']
)


In [71]:
from PIL import Image
import os

bad_images = []
root = "/content/train_data/train_data"  # your actual train directory

for fname in os.listdir(root):
    fpath = os.path.join(root, fname)
    try:
        img = Image.open(fpath)
        img.verify()   # check integrity
    except Exception as e:
        bad_images.append(fpath)

len(bad_images), bad_images[:10]


(0, [])

In [72]:
for f in bad_images:
    os.remove(f)

print("Removed", len(bad_images), "bad images.")


Removed 0 bad images.


In [73]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generators(train_df, val_df, test_df):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        zoom_range=0.1
    )

    test_val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='filepath',
        y_col='label',
        target_size=(160, 160),
        class_mode='raw',
        batch_size=16,
        shuffle=True
    )

    val_generator = test_val_datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col='filepath',
        y_col='label',
        target_size=(160, 160),
        class_mode='raw',
        batch_size=16,
        shuffle=False
    )

    test_generator = test_val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='filepath',
        y_col=None,
        target_size=(160, 160),
        class_mode=None,
        batch_size=16,
        shuffle=False
    )

    return train_generator, val_generator, test_generator


train_gen, val_gen, test_gen = create_generators(train_df, val_df, test)


Found 63959 validated image filenames.
Found 15990 validated image filenames.
Found 5540 validated image filenames.


/usr/local/lib/python3.12/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(


In [74]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2

IMG_SIZE = (160, 160)

base_model = MobileNetV2(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

model_mnv2 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(1, activation="sigmoid")
])

model_mnv2.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model_mnv2.summary()

history_mnv2 = model_mnv2.fit(
    train_gen,
    epochs=5,
    validation_data=val_gen
)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 600s 146ms/step - accuracy: 0.8575 - loss: 0.3265 - val_accuracy: 0.9031 - val_loss: 0.2330
Epoch 2/5
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 555s 139ms/step - accuracy: 0.9088 - loss: 0.2215 - val_accuracy: 0.9181 - val_loss: 0.2001
Epoch 3/5
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 548s 137ms/step - accuracy: 0.9190 - loss: 0.1962 - val_accuracy: 0.9197 - val_loss: 0.1930
Epoch 4/5
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 555s 139ms/step - accuracy: 0.9263 - loss: 0.1814 - val_accuracy: 0.9131 - val_loss: 0.2184
Epoch 5/5
3998/3998 ━━━━━━━━━━━━━━━━━━━━ 561s 139ms/step - accuracy: 0.9318 - loss: 0.1689 - val_accuracy: 0.9133 - val_loss: 0.2208


In [77]:
print("Final Training Accuracy:", history_mnv2.history["accuracy"][-1])
print("Final Validation Accuracy:", history_mnv2.history["val_accuracy"][-1])


Final Training Accuracy: 0.9306743144989014
Final Validation Accuracy: 0.9132583141326904
